In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import argparse
import os
import sys
import jax
import jax.numpy as jnp
# import jax.random
# from gymnasium import spaces
from tqdm import tqdm

from klax import lipschitz_l1_jax, triangular
from rl_environments import Vandelpol, Test2, LinearLQR
# from rl_environments import LDSEnv, InvertedPendulum, CollisionAvoidanceEnv, Vandelpol
from rsm_learner import Learner
from rsm_verifier import Verifier
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
parser = argparse.ArgumentParser(
    prog='System Normalization for Neural Certificates',
    description='Learning a reach-avoid supermartingale neural network with normalization techniques',
    epilog='By Musan (Hao Wu) @ SKLCS, Institute of Software, UCAS'
)

parser.add_argument("--debug", default=True, type=bool)
# problem formulation    
# parser.add_argument("--env", default="vandelpol", help='system')
parser.add_argument("--env", default="test2", help='system')
parser.add_argument("--timeout", default=60, type=int, help='max time limit in minutes') 
parser.add_argument("--reach_prob", default=0.8, type=float, help='reach-avoid probability')

parser.add_argument("--project", default=False, type=bool, help='Nomalize the system')

# neural network and training
parser.add_argument("--hidden", default=128, type=int, help='hidden neurons in each layer')
parser.add_argument("--num_layers", default=2, type=int, help='number of hidden layers')
# learner 
parser.add_argument("--continue_rsm", type=int, default=0, help='use an existing network')

# verifier
parser.add_argument("--eps", default=0.01, type=float) # ???
parser.add_argument("--lip", default=0.1, type=float) # ???

 

# parser.add_argument("--p_lip", default=0.0, type=float) 
parser.add_argument("--l_lip", default=10.0, type=float) # ???
parser.add_argument("--fail_check_fast", type=int, default=0)
parser.add_argument("--grid_factor", default=1.0, type=float)
parser.add_argument("--batch_size", default=512, type=int)
# parser.add_argument("--ppo_iters", default=50, type=int)
# parser.add_argument("--policy", default="policies/lds0_zero.jax")
# parser.add_argument("--train_p", type=int, default=1)
parser.add_argument("--square_l_output", default=True)
parser.add_argument("--jitter_grid", type=int, default=0)
parser.add_argument("--soft_constraint", type=int, default=1)
parser.add_argument("--gamma_decrease", default=1.0, type=float)
parser.add_argument("--debug_k0", action="store_true")
parser.add_argument("--gen_plot", action="store_true")
parser.add_argument("--no_refinement", action="store_true")
parser.add_argument("--plot", action="store_true")
parser.add_argument("--small_mem", action="store_true")
sys.argv = ['test.ipynb', '--env', 'LinearLQR']
args = parser.parse_args()

In [25]:
# env = Vandelpol()
# env = Test2()
env = LinearLQR()
learner = Learner(
    env=env,
    l_hidden=[args.hidden] * args.num_layers,
    l_lip=args.l_lip,
    eps=args.eps,
    gamma_decrease=args.gamma_decrease,
    reach_prob=args.reach_prob,
    softplus_l_output=args.square_l_output,
    debug = args.debug,
)

In [26]:
verifier = Verifier(
    learner=learner,
    env=env,
    batch_size=args.batch_size,
    reach_prob=args.reach_prob,
    fail_check_fast=bool(args.fail_check_fast),
    grid_factor=args.grid_factor,
    small_mem=args.small_mem,
    debug = args.debug,
)

In [27]:
from rsm_loop import RSMLoop
loop = RSMLoop( 
        learner,
        verifier,
        env,
        lip_factor=args.lip,
        plot=args.plot,
        jitter_grid=bool(args.jitter_grid),
        soft_constraint=bool(args.soft_constraint),
        debug = args.debug,
    )


loop.plot_l(f"plots/{args.env}_start.png")

In [21]:
loop.run(args.timeout*60)


## Iteration 0 (0:00 elapsed) ##












































































































































































































Train: loss=4.05, dec_loss=1.73, violations=0.425: 100%|██████████| 100/100 [00:37<00:00,  2.70it/s]


Trained on 40000 samples, start_loss=25.6, end_loss=4.05, start_violations=0.514, end_violations=0.425 in 37.1 seconds



100%|██████████| 3/3 [00:00<00:00, 137.10it/s]




100%|██████████| 77/77 [00:00<00:00, 215.37it/s]


lipschitz_k=33.01280227661133 (without delta)
delta=0.010050251256281407
K=0.3317869575538827 (with delta)
Checking GRID of size 200


...

100%|██████████| 77/77 [00:00<00:00, 258.31it/s]


lipschitz_k=29.83112113952637 (without delta)
delta=0.010050251256281407
K=0.29981026270880773 (with delta)
Checking GRID of size 200


100%|██████████| 1/1 [00:01<00:00,  1.92s/it]

violations=21359
hard_violations=21359
21359/40000 violated decrease condition
21359/40000 hard violations
Train buffer len: 458825
Grid runtime=2.26 s
info= {'ds_size': 437466, 'lipschitz_k': 29.83112113952637, 'K_f': 1.08, 'K_l': 27.621408462524414, 'iter': 19, 'runtime': 3382.314952135086, 'delta': 0.010050251256281407, 'K': 0.29981026270880773, 'avg_increase': 0, 'dec_violations': '21359/40000', 'hard_violations': '21359/40000'}
Timeout!


False

In [22]:
devices = jax.devices()
print(devices)

[CpuDevice(id=0)]
